In [8]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("responses.csv")

AttributeError: module 'pandas' has no attribute 'read'

In [ ]:
df.isnull().sum()

Timestamp                                                                                                                             0
What is the most visited cafeteria for you?                                                                                           1
What time visit the cafeteria most often?                                                                                             7
How often do you visit the cafeteria near you?                                                                                       71
Which day of the week do you visit the cafeteria most often?                                                                          2
For which kind of meals do you visit the cafeteria?                                                                                   0
In what kind of weather do you most often visit the cafeteria                                                                         0
What is the most common time gap (in hours) betw

In [ ]:
from sklearn.preprocessing import LabelEncoder
# Clean column names and values
df.columns = df.columns.str.strip().str.replace('\n', ' ')
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].astype(str).str.strip()

# Convert timestamp and visit time
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['Visit Time'] = pd.to_datetime(df['What time visit the cafeteria most often?'], errors='coerce').dt.time
df.dropna(subset=['What is the most visited cafeteria for you?', 'Visit Time'], inplace=True)

# Extract hour from time
df['Hour'] = pd.to_datetime(df['Visit Time'].astype(str), format='%H:%M:%S', errors='coerce').dt.hour

# Encode cafeteria names
le = LabelEncoder()
df['Cafeteria_Label'] = le.fit_transform(df['What is the most visited cafeteria for you?'])

C:\Users\avata\AppData\Local\Temp\ipykernel_19916\143032697.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Visit Time'] = pd.to_datetime(df['What time visit the cafeteria most often?'], errors='coerce').dt.time


In [ ]:
df.head()

,Timestamp,What is the most visited cafeteria for you?,What time visit the cafeteria most often?,How often do you visit the cafeteria near you?,Which day of the week do you visit the cafeteria most often?,For which kind of meals do you visit the cafeteria?,In what kind of weather do you most often visit the cafeteria,What is the most common time gap (in hours) between your visit to the cafeteria and your next class? Enter only a numeric value.,What do you feel is the most busiest time of the cafeterias you visit,Visit Time,Hour,Cafeteria_Label
0,2025-03-24 12:52:48.564,Techno,18:30:00,nan,Friday,Snacks,Sunny/Clear,nan,nan,18:30:00,18,5
1,2025-03-24 12:54:13.461,IC,19:00:00,nan,Sunday,Snacks,Cloudy,nan,nan,19:00:00,19,1
2,2025-03-24 12:56:23.550,IC,17:30:00,nan,Saturday,Snacks,Sunny/Clear,0,nan,17:30:00,17,1
4,2025-03-24 12:59:43.225,IC,19:00:00,nan,Thursday,Snacks,Cloudy,1,nan,19:00:00,19,1
5,2025-03-24 13:04:44.633,IC,12:50:00,nan,Friday,Snacks,Sunny/Clear,0.167,nan,12:50:00,12,1


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

In [ ]:
X = df[['Hour']]  # Features: Hour of visit
y = df['Cafeteria_Label']  # Target: Cafeteria

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model using Naive Bayes
model = MultinomialNB()
model.fit(X_train, y_train)
y_pred=model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.625


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB

# Define model
model = MultinomialNB()

# Define parameter grid
param_grid = {
    'alpha': [0.01, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0]
}

# GridSearchCV setup
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')

# Fit to training data
grid_search.fit(X_train, y_train)

# Best parameters and score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validated accuracy:", grid_search.best_score_)

# Use best model to predict
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Accuracy on test set
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f"Test accuracy with best model: {accuracy * 100:.2f}%")


c:\Users\avata\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best parameters: {'alpha': 0.01}
Best cross-validated accuracy: 0.5089743589743589
Test accuracy with best model: 62.50%


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Define model
model = MultinomialNB()

# Define parameter grid with more options
param_grid = {
    'alpha': [0.01, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0],
    'fit_prior': [True, False]
}

# Set up GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Fit model on training data
grid_search.fit(X_train, y_train)

# Best params and score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validated accuracy:", grid_search.best_score_)

# Use the best model for predictions
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Accuracy on test data
accuracy = accuracy_score(y_test, y_pred)
print(f"Test accuracy with tuned model: {accuracy * 100:.2f}%")


c:\Users\avata\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best parameters: {'alpha': 0.01, 'fit_prior': True}
Best cross-validated accuracy: 0.5089743589743589
Test accuracy with tuned model: 62.50%


In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Split your data (assuming X and y are already defined)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Dictionary to hold models and their param grids
models = {
    "MultinomialNB": {
        'model': MultinomialNB(),
        'params': {
            'alpha': [0.01, 0.1, 0.5, 1.0, 2.0],
            'fit_prior': [True, False]
        }
    },
    "LogisticRegression": {
        'model': LogisticRegression(max_iter=1000),
        'params': {
            'C': [0.01, 0.1, 1, 10],
            'solver': ['liblinear', 'lbfgs']
        }
    },
    "RandomForest": {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [50, 100],
            'max_depth': [None, 5, 10],
            'min_samples_split': [2, 5]
        }
    },
    "SVM": {
        'model': Pipeline([
            ('scaler', StandardScaler()),
            ('svc', SVC())
        ]),
        'params': {
            'svc__C': [0.1, 1, 10],
            'svc__kernel': ['linear', 'rbf']
        }
    }
}

# Run grid search for each model
best_models = {}

for name, m in models.items():
    print(f"\nTraining and tuning {name}...")
    grid = GridSearchCV(m['model'], m['params'], cv=5, scoring='accuracy', n_jobs=-1)
    grid.fit(X_train, y_train)
    
    print(f"Best parameters for {name}: {grid.best_params_}")
    print(f"Best CV accuracy: {grid.best_score_:.4f}")
    
    # Test accuracy
    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test)
    test_acc = accuracy_score(y_test, y_pred)
    print(f"Test accuracy: {test_acc:.4f}")
    
    best_models[name] = {
        'model': best_model,
        'cv_accuracy': grid.best_score_,
        'test_accuracy': test_acc
    }



Training and tuning MultinomialNB...
Best parameters for MultinomialNB: {'alpha': 0.01, 'fit_prior': True}
Best CV accuracy: 0.5090
Test accuracy: 0.6250

Training and tuning LogisticRegression...


c:\Users\avata\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\avata\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best parameters for LogisticRegression: {'C': 0.01, 'solver': 'lbfgs'}
Best CV accuracy: 0.6744
Test accuracy: 0.7500

Training and tuning RandomForest...


c:\Users\avata\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Best parameters for RandomForest: {'max_depth': None, 'min_samples_split': 5, 'n_estimators': 100}
Best CV accuracy: 0.6077
Test accuracy: 0.7500

Training and tuning SVM...
Best parameters for SVM: {'svc__C': 1, 'svc__kernel': 'rbf'}
Best CV accuracy: 0.6410
Test accuracy: 0.7500


c:\Users\avata\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
